# Submit a public workflow

This notebook shows how to use the WorkflowHub and LifeMonitor APIs to:

* Pick a workflow from WorkflowHub
* Submit it to LifeMonitor as a public workflow
* Query LifeMonitor about its testing status

In [1]:
import requests
import os
from collections import Counter

In [2]:
LM_BASE_URL = "https://api.lifemonitor.eu"
LM_API_KEY = os.environ["LM_API_KEY"]
WH_BASE_URL = "https://workflowhub.eu"

In [3]:
LM_SESSION = requests.Session()
WH_SESSION = requests.Session()
WH_SESSION.headers.update({
    "Content-type": "application/vnd.api+json",
    "Accept": "application/vnd.api+json",
    "Accept-Charset": "ISO-8859-1",
})

In [4]:
def lm_get(endpoint):
    response = LM_SESSION.get(f"{LM_BASE_URL}/{endpoint.lstrip('/')}")
    response.raise_for_status()
    return response.json()

def wh_get(endpoint):
    response = WH_SESSION.get(f"{WH_BASE_URL}/{endpoint.lstrip('/')}")
    response.raise_for_status()
    return response.json()["data"]

No auth yet: get public workflows

In [5]:
public_workflows = lm_get("/workflows")
public_workflows

{'items': [],
 'links': {'self': 'https://api.lifemonitor.eu/workflows'},
 'meta': {'api_version': '0.3.0',
  'base_url': 'https://api.lifemonitor.eu',
  'resource': '/workflows'}}

Update the session with auth info

In [6]:
LM_SESSION.headers.update({"ApiKey": LM_API_KEY})

Get the list of registries supported by LifeMonitor

In [7]:
registries = lm_get("/registries")["items"]
registries

[{'links': None,
  'name': 'wfhub',
  'type': 'seek',
  'uri': 'https://workflowhub.eu',
  'uuid': 'b556d831-a393-43bd-8047-98505b7fc20c'}]

In [8]:
registry = registries[0]
assert registry["uri"] == WH_BASE_URL

Look for a workflow in WorkflowHub. We're going to pick one from the [IWC project](https://github.com/galaxyproject/iwc). IWC workflows are tested with GitHub Actions, and provide information about such testing via [Workflow Testing RO-Crate](https://crs4.github.io/life_monitor/workflow_testing_ro_crate) metadata files (see, for instance, https://github.com/iwc-workflows/sars-cov-2-consensus-from-variation).

In [9]:
wh_projects = wh_get("/projects")
wh_proj_map = {_["attributes"]["title"]: _["id"] for _ in wh_projects}
iwc_proj_id = wh_proj_map["iwc"]
iwc_proj_id

'33'

Get information on IWC workflows on WorkflowHub

In [10]:
iwc_proj = wh_get(f"/projects/{iwc_proj_id}")
iwc_wf_ids = [_["id"] for _ in iwc_proj["relationships"]["workflows"]["data"]]
wh_workflows = wh_get("/workflows")
wh_wf_map = {_["id"]: _["attributes"]["title"] for _ in wh_workflows}
for id_ in iwc_wf_ids:
    print(id_, wh_wf_map[id_])

109 sars-cov-2-variation-reporting/COVID-19-VARIATION-REPORTING
110 sars-cov-2-pe-illumina-artic-variant-calling/COVID-19-PE-ARTIC-ILLUMINA
111 COVID-19: variation analysis of ARTIC ONT data
112 sars-cov-2-se-illumina-wgs-variant-calling/COVID-19-SE-WGS-ILLUMINA
113 sars-cov-2-pe-illumina-wgs-variant-calling/COVID-19-PE-WGS-ILLUMINA
137 parallel-accession-download/main
138 sars-cov-2-consensus-from-variation/COVID-19-CONSENSUS-CONSTRUCTION
155 SARS-CoV-2 Illumina Amplicon pipeline - iVar based


Let's pick `"sars-cov-2-consensus-from-variation/COVID-19-CONSENSUS-CONSTRUCTION"` and submit it to LifeMonitor.

In [11]:
wh_wf_id = "138"
wh_wf = wh_get(f"/workflows/{wh_wf_id}")
payload = {
    "identifier": wh_wf_id,
    "name": wh_wf["attributes"]["title"],
    "version": str(wh_wf["attributes"]["latest_version"]),
    "public": True,
}
payload

{'identifier': '138',
 'name': 'sars-cov-2-consensus-from-variation/COVID-19-CONSENSUS-CONSTRUCTION',
 'version': '2',
 'public': True}

In [12]:
response = LM_SESSION.post(f"{LM_BASE_URL}/registries/{registry['uuid']}/workflows", json=payload)
response.raise_for_status()
data = response.json()
data

{'name': 'sars-cov-2-consensus-from-variation/COVID-19-CONSENSUS-CONSTRUCTION',
 'uuid': 'e95b2410-cdc4-0139-988f-005056ab5db4',
 'wf_version': '2'}

Get the newly posted workflow from LifeMonitor

In [13]:
lm_wf_id = data["uuid"]
workflow = lm_get(f"/workflows/{lm_wf_id}")
workflow

{'links': {'self': 'https://api.lifemonitor.eu/workflows/e95b2410-cdc4-0139-988f-005056ab5db4'},
 'meta': {'api_version': '0.3.0',
  'base_url': 'https://api.lifemonitor.eu',
  'created': '2021-11-03T14:28:19.915805',
  'modified': '2021-11-03T14:28:19.915811',
  'resource': '/workflows/e95b2410-cdc4-0139-988f-005056ab5db4'},
 'name': 'sars-cov-2-consensus-from-variation/COVID-19-CONSENSUS-CONSTRUCTION',
 'public': True,
 'registry': {'name': 'wfhub',
  'type': 'seek',
  'uri': 'https://workflowhub.eu',
  'uuid': 'b556d831-a393-43bd-8047-98505b7fc20c'},
 'subscriptions': [],
 'uuid': 'e95b2410-cdc4-0139-988f-005056ab5db4',
 'version': {'is_latest': True,
  'links': {'origin': 'https://workflowhub.eu/workflows/138?version=2'},
  'name': 'sars-cov-2-consensus-from-variation/COVID-19-CONSENSUS-CONSTRUCTION',
  'ro_crate': {'links': {'download': 'https://api.lifemonitor.eu/workflows/e95b2410-cdc4-0139-988f-005056ab5db4/rocrate/2/download',
    'metadata': 'https://api.lifemonitor.eu/workfl

Get the workflow's test suites

In [14]:
suites = lm_get(f"/workflows/{lm_wf_id}/suites")["items"]
len(suites)

1

In [15]:
suite = lm_get(f"/suites/{suites[0]['uuid']}")
suite

{'definition': {'test_engine': {'type': 'planemo', 'version': '>=0.74.6'}},
 'instances': [{'links': {'origin': 'https://github.com/galaxyproject/iwc/actions/workflows/workflow_test.yml'},
   'managed': False,
   'name': 'test1_1',
   'resource': 'repos/galaxyproject/iwc/actions/workflows/workflow_test.yml',
   'roc_instance': '#test1_1',
   'service': {'type': 'github',
    'url': 'https://api.github.com',
    'uuid': '5812bfda-cff0-46ac-9e79-5e211367eac1'},
   'uuid': '2bb4eccf-19a7-47cb-962e-f90030ab6d9e'}],
 'links': {'self': 'https://api.lifemonitor.eu/suites/f7b76107-2aea-428e-a002-501afd15c2de'},
 'meta': {'api_version': '0.3.0',
  'base_url': 'https://api.lifemonitor.eu',
  'resource': '/suites/f7b76107-2aea-428e-a002-501afd15c2de'},
 'roc_suite': '#test1',
 'uuid': 'f7b76107-2aea-428e-a002-501afd15c2de'}

There's only one suite. Get its instances (executions of the suite on CI services):

In [16]:
instances = lm_get(f"/suites/{suite['uuid']}/instances")["items"]
len(instances)

1

In [17]:
instance = lm_get(f"/instances/{instances[0]['uuid']}")
instance

{'links': {'origin': 'https://github.com/galaxyproject/iwc/actions/workflows/workflow_test.yml',
  'self': 'https://api.lifemonitor.eu/instances/2bb4eccf-19a7-47cb-962e-f90030ab6d9e'},
 'managed': False,
 'meta': {'api_version': '0.3.0',
  'base_url': 'https://api.lifemonitor.eu',
  'resource': '/instances/2bb4eccf-19a7-47cb-962e-f90030ab6d9e'},
 'name': 'test1_1',
 'resource': 'repos/galaxyproject/iwc/actions/workflows/workflow_test.yml',
 'roc_instance': '#test1_1',
 'service': {'type': 'github',
  'url': 'https://api.github.com',
  'uuid': '5812bfda-cff0-46ac-9e79-5e211367eac1'},
 'uuid': '2bb4eccf-19a7-47cb-962e-f90030ab6d9e'}

In [18]:
print(instance["links"]["origin"])

https://github.com/galaxyproject/iwc/actions/workflows/workflow_test.yml


There's only one instance, corresponding to the workflow's test execution on GitHub Actions. Get info on the latest builds for this instance:

In [19]:
latest_builds = lm_get(f"/instances/{instance['uuid']}/latest-builds")["items"]
len(latest_builds)

10

In [20]:
build = latest_builds[0]
build

{'build_id': '1401905791',
 'duration': 1000,
 'instance': {'links': {'origin': 'https://github.com/galaxyproject/iwc/actions/workflows/workflow_test.yml'},
  'managed': False,
  'name': 'test1_1',
  'resource': 'repos/galaxyproject/iwc/actions/workflows/workflow_test.yml',
  'roc_instance': '#test1_1',
  'service': {'type': 'github',
   'url': 'https://api.github.com',
   'uuid': '5812bfda-cff0-46ac-9e79-5e211367eac1'},
  'uuid': '2bb4eccf-19a7-47cb-962e-f90030ab6d9e'},
 'links': {'origin': 'https://github.com/galaxyproject/iwc/actions/runs/1401905791'},
 'status': 'failed',
 'suite_uuid': 'f7b76107-2aea-428e-a002-501afd15c2de',
 'timestamp': '1635583833'}

In [21]:
Counter(_["status"] for _ in latest_builds)

Counter({'failed': 5, 'passed': 4, 'aborted': 1})

In [22]:
print(build["links"]["origin"])

https://github.com/galaxyproject/iwc/actions/runs/1401905791


Close the current sessions, then make an unauthenticated request to check again for public workflows.

In [23]:
WH_SESSION.close()
LM_SESSION.close()
response = requests.get(f"{LM_BASE_URL}/workflows")
response.raise_for_status
response.json()["items"]

[{'latest_version': '2',
  'name': 'sars-cov-2-consensus-from-variation/COVID-19-CONSENSUS-CONSTRUCTION',
  'public': True,
  'uuid': 'e95b2410-cdc4-0139-988f-005056ab5db4'}]